# <center>Bangla Handwritten Math Solver (GUI)<center> 

## Importing Libraries 

In [1]:
from keras.models import load_model
from tkinter import *
from tkinter.ttk import *
from PIL import ImageGrab
import cv2
import imutils
from imutils.contours import sort_contours
import numpy as np
import matplotlib.pyplot as plt

## Initializing Canvas

In [2]:
res = ""
pre = [None, None]
bs = 5.5

root = Tk()
root.title("Bangla Handwriting Math Solver")
root.resizable(False, False)
# root.overrideredirect(True) # turns off title bar

# root size we want to create
root_height = 600
root_width = 1024

# getting the full screen height and width
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# calculating the geometry padding
x_cordinate = int((screen_width/2) - (root_width/2))
y_cordinate = int((screen_height/2) - (root_height/2))

# canvas size 
canvas_width = root_width - 52
canvas_height = root_height - 150

root.geometry("{}x{}+{}+{}".format(root_width, root_height, x_cordinate, y_cordinate))
c = Canvas(root, bd=3, relief="ridge", bg='white', height=canvas_height, width=canvas_width)
c.grid(row=0, column=0, columnspan=3, padx=20, pady=10)

## Function for closing window

In [3]:
def close():
    root.destroy()

## Function for clearing the canvas

In [4]:
def clear():
    c.delete("all")

## Function for putting a point on the canvas

In [5]:
def putPoint(e):
    c.create_oval(e.x - bs, e.y - bs, e.x + bs, e.y + bs, outline='black', fill='black')
    pre = [e.x, e.y]

## Function for drawing on the canvas

In [6]:
def paint(e):
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
    pre = [e.x, e.y]

## Function for getting the image from the canvas

In [7]:
def get_image():
    print('Processing image from the canvas...')
    label['text'] = 'Processing image from the canvas...'
    #
    x, y = (c.winfo_rootx(), c.winfo_rooty())
    width, height = (c.winfo_width(), c.winfo_height())
    a, b, c, d = (x, y, x+width, y+height)
    #
    img = ImageGrab.grab()
    img.save("output/1_full-screen.png")
    img = img.crop((a + 76, b + 48, c + 313, d + 154))
    img.save("output/2_drawn-image.png")
    print('Image saved!')
    label['text'] = 'Image saved!'
    return True

## Functions for getting the equation and solve

In [9]:
# Function for solving the prediction
def get_equation_and_solve():
    print('Solving the equation...')
    label['text'] = 'Loading the model...'
    model = load_model('math_symbol_and_digit_recognition.h5')
    chars = []
    img = cv2.imread('output/2_drawn-image.png')
    img = cv2.resize(img, (canvas_width, canvas_height))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(img_gray, 30, 150)
    contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    print('Number of contours found: ', len(contours))
    contours = sort_contours(contours, method="left-to-right")[0]
    labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'add', 'div', 'mul', 'sub']

    for i, c in enumerate(contours):
        print('Processing the image...: ', str(i+1))
        (x, y, w, h) = cv2.boundingRect(c)
        print('x: ', x, 'y: ', y, 'w: ', w, 'h: ', h)
        # if x > 0 and y > 0 and w >= 5 and h >= 25:
        if x > 0 and y > 0 and w > 5:  # cheaking weather any garbage value detecting
            roi = img_gray[y:y+h, x:x+w]
            thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            (th, tw) = thresh.shape
            if tw > th:
                thresh = imutils.resize(thresh, width=32)
            if th > tw:
                thresh = imutils.resize(thresh, height=32)
            (th, tw) = thresh.shape
            dx = int(max(0, 32 - tw)/2.0)
            dy = int(max(0, 32 - th) / 2.0)
            padded = cv2.copyMakeBorder(thresh, top=dy, bottom=dy, left=dx, right=dx, borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0))
            padded = cv2.resize(padded, (32, 32))
            padded = np.array(padded)
            padded = padded/255.
            padded = np.expand_dims(padded, axis=0)
            padded = np.expand_dims(padded, axis=-1)
            pred = model.predict(padded)
            pred = np.argmax(pred, axis=1)
            label = labels[pred[0]]
            print('>>>>The {} no word is : {}'.format(i, label))
            chars.append(label)
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(img, label, (x-5, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))

    plt.figure(figsize=(10, 10))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.savefig('output/3_system_prediction.png')

    e = ''
    print('Equation: {}', chars)
    for i in chars:
        if i=='add':
            e += '+'
        elif i=='sub':
            e += '-'
        elif i=='mul':
            e += '*'
        elif i=='div':
            e += '/'
        else:
            e += i
    v = eval(e)
    print('V Result: {}', v)
    print('E Result: {}', e)

    self.label['text'] = 'The result is: {} : {}'.format(e, v) 
    print('Value of the expression {} : {}'.format(e, v))
    pass

## Functions for proccessing image and equation

In [ ]:
def solve():
    print('Starting the work...')
    success = get_image()
    if success:
        get_equation_and_solve()
    else:
        print("Error in fetching image!")
        label['text'] = 'Error in fetching image!'

## Functions for config canvas

In [ ]:
# Create label
label = Label(root, text = "Draw your mathematical term here...👆👆", )
label.config(font =("Courier", 14))
label.grid(row=1, column=0, columnspan=3)

# Create label blank for spacing
space = Label(root, text = "", )
space.config(font =("Courier", 14))
space.grid(row=2, column=0, columnspan=3)

style = Style()

''' Button 1: Exit'''
style.configure('E.TButton', font = ('calibri', 15, 'bold', 'underline'), foreground = 'red')
exit_btn = Button(root, text = 'Quit !', style = 'E.TButton', command = close)
exit_btn.grid(row = 3, column = 0)

''' Button 2: Clear'''
style.configure('C.TButton', font = ('calibri', 15, 'bold', 'underline'), foreground = 'blue')
calculate_btn = Button(root, text = 'Clear', style = 'C.TButton', command = clear)
calculate_btn.grid(row = 3, column = 1)

''' Button 3: Solve'''
style.configure('S.TButton', font = ('calibri', 15, 'bold', 'underline'), foreground = 'green')
exit_btn = Button(root, text = 'Solve', style = 'S.TButton', command = solve)
exit_btn.grid(row = 3, column = 2)

c.bind("<Button-1>", putPoint)
# c.bind("<ButtonRelease-1>", getResult)
c.bind("<B1-Motion>", paint)

## Executing function

In [12]:
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\sabik\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\sabik\AppData\Local\Temp\ipykernel_12876\1510092135.py", line 2, in paint
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
UnboundLocalError: local variable 'pre' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

KeyboardInterrupt: 

# <center>Bangla Handwritten Math Solver (GUI)<center> 

In [1]:
from tkinter import *
from tkinter.ttk import *
from PIL import ImageGrab

In [2]:
res = ""
pre = [None, None]
bs = 5.5

root = Tk()
root.title("Bangla Handwriting Math Solver")
root.resizable(False, False)
# root.overrideredirect(True) # turns off title bar

# root size we want to create
root_height = 600
root_width = 1024

# getting the full screen height and width
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# calculating the geometry padding
x_cordinate = int((screen_width/2) - (root_width/2))
y_cordinate = int((screen_height/2) - (root_height/2))

root.geometry("{}x{}+{}+{}".format(root_width, root_height, x_cordinate, y_cordinate))
c = Canvas(root, bd=3, relief="ridge", bg='white', height=root_height - 150, width=root_width - 52)
c.grid(row=0, column=0, columnspan=3, padx=20, pady=10)

# Create label
label = Label(root, text = "Draw your mathematical term here...👆👆", )
label.config(font =("Courier", 14))
label.grid(row=1, column=0, columnspan=3)

# Create label blank for spacing
space = Label(root, text = "", )
space.config(font =("Courier", 14))
space.grid(row=2, column=0, columnspan=3)

style = Style()

''' Button 1: Exit'''
style.configure('E.TButton', font = ('calibri', 15, 'bold', 'underline'), foreground = 'red')
exit_btn = Button(root, text = 'Quit !', style = 'E.TButton', command = close)
exit_btn.grid(row = 3, column = 0)

''' Button 2: Clear'''
style.configure('C.TButton', font = ('calibri', 15, 'bold', 'underline'), foreground = 'blue')
calculate_btn = Button(root, text = 'Clear', style = 'C.TButton', command = clear)
calculate_btn.grid(row = 3, column = 1)

''' Button 3: Solve'''
style.configure('S.TButton', font = ('calibri', 15, 'bold', 'underline'), foreground = 'green')
exit_btn = Button(root, text = 'Solve', style = 'S.TButton', command = solve)
exit_btn.grid(row = 3, column = 2)

c.bind("<Button-1>", putPoint)
# c.bind("<ButtonRelease-1>", getResult)
c.bind("<B1-Motion>", paint)
root.mainloop()

NameError: name 'close' is not defined

In [ ]:
# Function for closing window
def close():
    root.destroy()

In [ ]:
# Function for clearing the canvas
def clear():
    c.delete("all")

In [ ]:
# Function for putting a point on the canvas
def putPoint(e):
    c.create_oval(e.x - bs, e.y - bs, e.x + bs, e.y + bs, outline='black', fill='black')
    pre = [e.x, e.y]

In [ ]:
# Function for drawing on the canvas
def paint(e):
    c.create_line(pre[0], pre[1], e.x, e.y, width=bs * 2, fill='black', capstyle=ROUND, smooth=TRUE)
    pre = [e.x, e.y]

In [ ]:
def solve():
    print('Starting the work...')
    success = get_image()
    if success:
        get_solve()

In [ ]:
# Function for getting the image from the canvas
def get_image():
    print('Processing image from the canvas...')
    #
    x, y = (c.winfo_rootx(), c.winfo_rooty())
    width, height = (c.winfo_width(), c.winfo_height())
    a, b, c, d = (x, y, x+width, y+height)
    #
    img = ImageGrab.grab()
    img.save("full-screen.png")
    img = img.crop((a + 76, b + 48, c + 313, d + 154))
    img.save("drawn-image.png")
    print('Image saved!')
    return True

In [ ]:
# Function for solving the prediction
def get_solve():
    print('Solving the equation...')
    pass

In [ ]:
#Executing function
